# Working with dataframes in python using Pandas

Pandas is built on top of the **NumPy** package, meaning a lot of the structure of NumPy is used or replicated in Pandas. Data in pandas is often used to feed statistical analysis in **SciPy**, plotting functions from **Matplotlib**, and machine learning algorithms in **Scikit-learn**.

> Note: this lesson will just scratch the surface of what is possible with Pandas. It's focused on most common commands and operations and should be seen as a starting point for further investigation.

In [ ]:
# Pandas is usually imported like this:
import pandas as pd
# we can override default options of pandas, such as automatic column and row hiding when the dataset is to large, with the following commands:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

## Core components of pandas

The primary two components of pandas are the `Series` and `DataFrame`.

A `Series` is essentially a column, and a `DataFrame` is a multi-dimensional table made up of a collection of `Series`.

## Creating DataFrames from scratch

There are many ways to create a `DataFrame` from scratch, but a great option is to just use a simple `dict`.

In [ ]:
data = {
    'apples': [3, 2, 0, 1], 
    'oranges': [0, 3, 7, 2]
}

purchases = pd.DataFrame(data)

purchases

The `Index` of this `DataFrame` was given to us on creation as the numbers 0-3, but we could also create our own when we initialize the `DataFrame`.

Let's have customer names as our index:

In [ ]:
purchases = pd.DataFrame(data, index=['June', 'Robert', 'Lily', 'David'])

purchases

So now we could locate a customer's order by using their name:

In [ ]:
purchases.loc['June']

## How to read in data

It’s quite simple to load data from various file formats into a `DataFrame`. In the following examples we'll keep using our apples and oranges data, but this time it's coming from various files.

### Reading data from csv

In [ ]:
df = pd.read_csv('purchases.csv')

df

CSVs don't have indexes like our `DataFrames`, so all we need to do is just designate the `index_col` when reading:

In [ ]:
# we're setting the index to be column zero (the customers names)
df = pd.read_csv('purchases.csv', index_col=0)

df

### Reading data from json

In [ ]:
df = pd.read_json('purchases.json')

df

### Reading data from database

If you’re working with data from a SQL database you need to first establish a connection using an appropriate Python library, then pass a query to pandas. Here we'll use SQLite to demonstrate.

In [ ]:
import sqlite3

con = sqlite3.connect("database.db")

df = pd.read_sql_query("SELECT * FROM purchases", con)

df

Just like with CSVs, we could pass `index_col='index'`, but we can also set an index after-the-fact:

In [ ]:
df = df.set_index('index')

df

### Reading from Excel

In [ ]:
df = pd.read_excel('purchases.xlsx', index_col=0)

df

## Converting back to a CSV, JSON, Database or Excel

In [ ]:
df.to_csv('new_purchases.csv')

df.to_json('new_purchases.json')

df.to_sql('new_purchases', con, if_exists='replace')

df.to_excel('new_purchases.xlsx')

## Most important DataFrame operations

`DataFrames` possess hundreds of methods and other operations that are crucial to any analysis. As a beginner, you should know the operations that perform simple transformations of your data and those that provide fundamental statistical analysis.

Let's load in the IMDB movies dataset to begin:

In [ ]:
movies_df = pd.read_csv("IMDB-Movie-Data.csv", index_col="Title")

### Viewing your data

The first thing to do when opening a new dataset is print out a few rows to keep as a visual reference. We accomplish this with `head`:

In [ ]:
movies_df.head() 

#movies_df.head(10)  # you can specify how many rows you want to see by passing a number

To see the last five rows use `tail`, which also accepts a number, and in this case we printing the bottom two rows:

In [ ]:
movies_df.tail()

### Getting info about your data

`info` should be one of the very first commands you run after loading your data:

In [ ]:
movies_df.info()

Another fast and useful attribute is `shape`, which outputs just a tuple of (rows, columns):

In [ ]:
movies_df.shape

### Handling duplicates

This dataset does not have duplicate rows, but it is always important to verify you aren't aggregating duplicate rows.

To demonstrate, let's simply just double up our movies DataFrame by appending it to itself:

In [ ]:
temp_df = pd.concat([movies_df, movies_df])

temp_df.shape

Now we can try dropping duplicates:

In [ ]:
no_dups = temp_df.drop_duplicates()

#no_dups.shape
temp_df.shape

Just like `append`, the `drop_duplicates()` method will also return a copy of your `DataFrame`, but this time with duplicates removed. Calling `shape` confirms we're back to the 1000 rows of our original dataset.

It's a little verbose to keep assigning `DataFrames` to the same variable like in this example. For this reason, pandas has the `inplace` keyword argument on many of its methods. Using `inplace=True` will modify the `DataFrame` object in place:

In [ ]:
temp_df.drop_duplicates(inplace=True)
temp_df.shape

Another important argument for `drop_duplicates()` is `keep`, which has three possible options:

* `first`: (default) Drop duplicates except for the first occurrence.
* `last`: Drop duplicates except for the last occurrence.
* `False`: Drop all duplicates.

Since we didn't define the `keep` argument in the previous example it was defaulted to `first`. This means that if two rows are the same pandas will drop the second row and keep the first row. Using `last` has the opposite effect: the first row is dropped.

`keep`, on the other hand, will drop all duplicates. If two rows are the same then both will be dropped. Watch what happens to `temp_df`:

In [ ]:
temp_df = pd.concat([movies_df, movies_df]) # make a new copy

temp_df.drop_duplicates(inplace=True, keep=False)

temp_df.shape

Since all rows were duplicates, `keep=False` dropped them all resulting in zero rows being left over. If you're wondering why you would want to do this, one reason is that it allows you to locate all duplicates in your dataset. When conditional selections are shown below you'll see how to do that.

### Column cleanup

Many times datasets will have verbose column names with symbols, upper and lowercase words, spaces, and typos. To make selecting data by column name easier we can spend a little time cleaning up their names.

Here's how to print the column names of our dataset:

In [ ]:
movies_df.columns

We can use the `rename` method to rename certain or all columns via a dict. We don't want parentheses, so let's rename those:

In [ ]:
movies_df.rename(columns={
        'Runtime (Minutes)': 'Runtime', 
        'Revenue (Millions)': 'Revenue_millions'
    }, inplace=True)

movies_df.columns

Excellent. But what if we want to lowercase all names? Instead of using `rename` we could also iterate the columns using a comprehension, like so:

In [ ]:
movies_df.columns = [col.lower() for col in movies_df.columns]

#columns = []
#for col in movies_df:
#     columns.append(col.lower())
    
#movies_df.columns = columns

movies_df.columns

It's a good idea to lowercase, remove special characters, and replace spaces with underscores if you'll be working with a dataset for some time.

### How to work with missing values

When exploring data, you’ll most likely encounter missing or null values, which are essentially placeholders for non-existent values. Most commonly you'll see Python's `None` or NumPy's `np.nan`, each of which are handled differently in some situations.

There are two options in dealing with nulls: 

1. Get rid of rows or columns with nulls
2. Replace nulls with non-null values, a technique known as **imputation**

Let's calculate to total number of nulls in each column of our dataset. The first step is to check which cells in our `DataFrame` are null:

In [ ]:
movies_df.isnull()

Notice `isnull` returns a `DataFrame` where each cell is either True or False depending on that cell's null status.

To count the number of nulls in each column we use an aggregate function for summing:

In [ ]:
movies_df.isnull().sum()

#### Removing rows or columns with null values

Data Scientists and Analysts regularly face the dilemma of dropping or imputing null values, and is a decision that requires intimate knowledge of your data and its context. Overall, removing null data is only suggested if you have a small amount of missing data.

Remove nulls is pretty simple:

In [ ]:
movies_df.dropna(axis=0)

This operation will delete any **row** with at least a single null value, but it will return a new DataFrame without altering the original one. You could specify `inplace=True` in this method as well.

So in the case of our dataset, this operation would remove 128 rows where `revenue_millions` is null and 64 rows where `metascore` is null. This obviously seems like a waste since there's perfectly good data in the other columns of those dropped rows. That's why we'll look at imputation next.

Other than just dropping rows, you can also drop columns with null values by setting `axis=1`:

In [ ]:
movies_df.dropna(axis=1)

In our dataset, this operation would drop the `revenue_millions` and `metascore` columns.

>**Side note**: What's with this `axis=1` parameter?

It's not immediately obvious where `axis` comes from and why you need it to be 1 for it to affect columns. To see why, just look at the `.shape` output:

In [ ]:
movies_df.shape

As we learned above, this is a tuple that represents the shape of the DataFrame, i.e. 1000 rows and 11 columns. Note that the *rows* are at index zero of this tuple and *columns* are at **index one** of this tuple. This is why `axis=1` affects columns. This comes from NumPy, and is a great example of why learning NumPy is worth your time.

#### Imputation

Imputation is a conventional feature engineering technique used to keep valuable data that have null values. 

There may be instances where dropping every row with a null value removes too big a chunk from your dataset, so instead we can impute that null with another value, usually the **mean** or the **median** of that column. 

Let's look at imputing the missing values in the `revenue_millions` column. First we'll extract that column into its own variable:

In [ ]:
revenue = movies_df['revenue_millions']

Using square brackets is the general way we select columns in a DataFrame. 

If you remember back to when we created DataFrames from scratch, the keys of the `dict` ended up as column names. Now when we select columns of a DataFrame, we use brackets just like if we were accessing a Python dictionary. 

`revenue` now contains a Series:

In [ ]:
revenue.head()

Slightly different formatting than a DataFrame, but we still have our `Title` index. 

We'll impute the missing values of revenue using the mean. Here's the mean value:

In [ ]:
revenue_mean = revenue.mean()

revenue_mean

With the mean, let's fill the nulls using `fillna()`:

In [ ]:
revenue.fillna(revenue_mean, inplace=True)

We have now replaced all nulls in `revenue` with the mean of the column. Notice that by using `inplace=True` we have actually affected the original `movies_df`:

In [ ]:
movies_df.isnull().sum()

Imputing an entire column with the same value like this is a basic example. It would be a better idea to try a more granular imputation by Genre or Director. 

For example, you would find the mean of the revenue generated in each genre individually and impute the nulls in each genre with that genre's mean.

Let's now look at more ways to examine and understand the dataset.

### Understanding your variables

Using `describe` on an entire DataFrame we can get a summary of the distribution of continuous variables:

In [ ]:
movies_df.describe()

Understanding which variables are continuous also comes in handy when thinking about the type of plot to use to represent your data visually. 

`describe` can also be used on a categorical variable to get the count of rows, unique count of categories, top category, and freq of top category:

In [ ]:
movies_df['genre'].describe()

In [ ]:
movies_df.head(10)

This tells us that the genre column has 207 unique values, the top value is Action/Adventure/Sci-Fi, which shows up 50 times (freq).

`value_counts` can tell us the frequency of all values in a column:

In [ ]:
movies_df['genre'].value_counts().head(10)

#### Relationships between continuous variables

By using the correlation method `.corr()` we can generate the relationship between each continuous variable:

In [ ]:
movies_df.corr(numeric_only=True)

#import numpy as np
#movies_df.select_dtypes(include=np.number).corr()

Correlation tables are a numerical representation of the bivariate relationships in the dataset. 

Positive numbers indicate a positive correlation — one goes up the other goes up — and negative numbers represent an inverse correlation — one goes up the other goes down. 1.0 indicates a perfect correlation. 

So looking in the first row, first column we see `rank` has a perfect correlation with itself, which is obvious. On the other hand, the correlation between `votes` and `revenue_millions` is 0.6. A little more interesting.

Examining bivariate relationships comes in handy when you have an outcome or dependent variable in mind and would like to see the features most correlated to the increase or decrease of the outcome. You can visually represent bivariate relationships with scatterplots (seen below in the plotting section). 

For a deeper look into data summarizations check out [Essential Statistics for Data Science](https://www.learndatasci.com/tutorials/data-science-statistics-using-python/).

Let's see more about manipulating DataFrames.

### Slicing, selecting, extracting

Up until now we've focused on some basic summaries of our data. We've learned about simple column extraction using single brackets, and we imputed null values in a column using `fillna`. Below are the other methods of slicing, selecting, and extracting you'll need to use constantly.

It's important to note that, although many methods are the same, DataFrames and Series have different attributes, so you'll need be sure to know which type you are working with or else you will receive attribute errors. 

Let's look at working with columns first.

#### By column

You already saw how to extract a column using square brackets like this:

In [ ]:
genre_col = movies_df['genre']

type(genre_col)

This will return a `Series`. To extract a column as a `DataFrame`, you need to pass a list of column names. In our case that's just a single column:

In [ ]:
genre_col = movies_df[['genre']]

type(genre_col)

Since it's just a list, adding another column name is easy:

In [ ]:
subset = movies_df[['genre', 'rating']]

subset.head()

Now we'll look at getting data by rows.

#### By rows

For rows, we have two options: 

- `loc` - **loc**ates by name
- `iloc`- **loc**ates by numerical **i**ndex

Remember that we are still indexed by movie Title, so to use `loc` we give it the Title of a movie:

In [ ]:
prom = movies_df.loc["Prometheus"]

prom

On the other hand, with `iloc` we give it the numerical index of Prometheus:

In [ ]:
prom = movies_df.iloc[1]

prom

`loc` and `iloc` can be thought of as similar to Python `list` slicing. To show this even further, let's select multiple rows.

How would you do it with a list? In Python, just slice with brackets like `example_list[1:4]`. It's works the same way in pandas:

In [ ]:
movie_subset = movies_df.loc['Prometheus':'Sing']

#movie_subset = movies_df.iloc[1:4]

movie_subset

One important distinction between using `loc` and `iloc` to select multiple rows is that `loc` includes the movie *Sing* in the result, but when using `iloc` we're getting rows 1:4 but the movie at index 4 (*Suicide Squad*) is not included. 

Slicing with `iloc` follows the same rules as slicing with lists, the object at the index at the end is not included.

#### Conditional selections
We’ve gone over how to select columns and rows, but what if we want to make a conditional selection? 

For example, what if we want to filter our movies DataFrame to show only films directed by Ridley Scott or films with a rating greater than or equal to 8.0?

To do that, we take a column from the DataFrame and apply a Boolean condition to it. Here's an example of a Boolean condition:

In [ ]:
condition = (movies_df['director'] == "Ridley Scott")

condition.head(10)

Similar to `isnull`, this returns a `Series` of `True` and `False` values: True for films directed by Ridley Scott and False for ones not directed by him. 

We want to filter out all movies not directed by Ridley Scott, in other words, we don’t want the False films. To return the rows where that condition is True we have to pass this operation into the DataFrame:

In [ ]:
movies_df[movies_df['director'] == "Ridley Scott"].head()

You can get used to looking at these conditionals by reading it like: 

> Select movies_df where movies_df director equals Ridley Scott

Let's look at conditional selections using numerical values by filtering the DataFrame by ratings:

In [ ]:
movies_df[movies_df['rating'] >= 8.6].head(3)

We can make some richer conditionals by using logical operators `|` for "or" and `&` for "and".

Let's filter the the DataFrame to show only movies by Christopher Nolan OR Ridley Scott:

In [ ]:
movies_df[(movies_df['director'] == 'Christopher Nolan') | (movies_df['director'] == 'Ridley Scott')].head()

We need to make sure to group evaluations with parentheses so Python knows how to evaluate the conditional.

Using the `isin` method we could make this more concise though:

In [ ]:
movies_df[movies_df['director'].isin(['Christopher Nolan', 'Ridley Scott'])].head()

Let's say we want all movies that were released between 2005 and 2010, have a rating above 8.0, but made below the 25th percentile in revenue.

Here's how we could do all of that:

In [ ]:
movies_df[
    ((movies_df['year'] >= 2005) & (movies_df['year'] <= 2010))
    & (movies_df['rating'] > 8.0)
    & (movies_df['revenue_millions'] < movies_df['revenue_millions'].quantile(0.25))
]

If you recall up when we used `describe` the 25th percentile for revenue was about 17.4, and we can access this value directly by using the `quantile` method with a float of 0.25.

So here we have only four movies that match that criteria.

#### Using query()

Pandas introduced the `query()` method in order to facilitate the data selection methods we've seen above.

Let's go through some of the selections we made above, using `query()`:

In [ ]:
movies_df.query('director == "Ridley Scott"')

#movies_df.query('rating >= 8.6')

#movies_df.query('director in ["Ridley Scott", "Christopher Nolan"]')

#movies_df.query('(year >= 2005 and year <= 2010) and rating > 8.0 \
#                and revenue_millions < revenue_millions.quantile(0.25)')

In [ ]:
# we can also use external variables inside query():
revenue_mean = movies_df.revenue_millions.mean()
movies_df.query('revenue_millions > @revenue_mean')

# we can also directly compare two columns. Since our dataframe lacks two comparable columns, 
# here's a ficticious example:
# my_df.query('sales > expenses')

In [ ]:
# finally, you can apply the query directly in the dataframe:
movies_df_copy = movies_df.copy()
movies_df_copy.query('rating >= 8.6', inplace=True)
movies_df_copy.shape

#movies_df_copy = movies_df.query('rating >= 8.6').copy()
#movies_df_copy.shape

### Sorting

Another common need is sorting the data. There are two different ways of sorting: one is sorting by column names and another by the dataframe's index (Title in our example).

So if we want to sort by Title, since it's an index we have to do it like this:

In [ ]:
# we're sorting by axis 0, i.e. by rows, in descending mode
movies_df.sort_index(axis = 0, ascending=False)

Look at what happens when we sort by axis 1, i.e. columns:

In [ ]:
movies_df.sort_index(axis = 1)

The columns got sorted and shifted positions.

Now if we wanted to sort by a column:

In [ ]:
movies_df.sort_values(by='director', ascending=True)

In [ ]:
# we can sort my multiple columns as well
movies_df.sort_values(by=['director', 'Title'], ascending=True)

# and specify a different sorting order for each column
#movies_df.sort_values(by=['director', 'Title'], ascending=[True, False])

### Vlookup / joins

It's common to have multiple datasources that you'll need to join, usually using a column that exists on both datasources. It's a operation similar to a `join` in an SQL database or a `vlookup` on Excel.

Let's load two datasets and join them on a mutual 'key' column. Note that **the 'key' must be an index in both dataframes**.

In [ ]:
df = pd.read_csv('purchases_join.csv')
df_2 = pd.read_csv('purchases_join_2.csv')

df.set_index('name', inplace=True)
df_2.set_index('name', inplace=True)
joined = df.join(df_2, on='name')

joined.head()

An alternative method, that doesn't need indexes, is the `merge` method:

In [ ]:
merged = pd.merge(df, df_2, on='name')

merged.head()

What if you have two datasets with the same columns, and only want to join the rows from both? Use the `concat` method:

In [ ]:
df = pd.read_csv('purchases_concat.csv')
df_2 = pd.read_csv('purchases_concat_2.csv')

# to use when datasets have no headers or different headers
#df = pd.read_csv('purchases_concat.csv', header=None)
#df_2 = pd.read_csv('purchases_concat_2.csv', header=None)

concatenated = pd.concat([df, df_2], ignore_index=True)

concatenated.head(10)

Notice the use of `ignore_index=True`: we're saying that we don't want the indexes from the dataframes we're concatenating, but instead have a new index built from the concatenation. Try to remove it to see the difference in the index.

### Pivot / groupby

It's also common the need to group and aggregate values based on one or more columns of your dataset. It's a operation similar to a `group by` in an SQL database or a `pivot` table in Excel.

In [ ]:
# comment and uncomment the examples below to see different group by's
df = pd.read_csv('orders.csv')

df.head(12)

In [ ]:
# count orders belonging to each user
df.groupby(['user_id']).agg({'order_id': 'count'})

# count orders and sum total value of orders by user
#df.groupby(['user_id']).agg({'order_id': 'count', 'total': 'sum'})

# same as above, but instead of sum we want the average order value for each user
#df.groupby(['user_id']).agg({'order_id': 'count', 'total': 'mean'})

# group orders by user_id
#df.groupby(['user_id', 'order_id']).sum()

In [80]:
#exporting the pivot table
pivot = df.groupby(['user_id', 'order_id']).sum()

pivot.to_csv('pivot.csv')
pivot.to_json('pivot.json')

<class 'pandas.core.frame.DataFrame'>


## Applying functions

It is possible to iterate over a `DataFrame` or `Series` as you would with a list, but doing so — especially on large datasets — is very slow.

An efficient alternative is to `apply` a function to the dataset. For example, we could use a function to convert movies with an 8.0 or greater to a string value of "good" and the rest to "bad" and use this transformed values to create a new column.

First we would create a function that, when given a rating, determines if it's good or bad:

In [ ]:
def rating_function(x):
    if x >= 8.0:
        return "good"
    else:
        return "bad"

Now we want to send the entire rating column through this function, which is what `apply()` does:

In [ ]:
movies_df["rating_category"] = movies_df["rating"].apply(rating_function)

movies_df.head(5)

The `.apply` method passes every value in the `rating` column through the `rating_function` and then returns a new `Series`. This `Series` is then assigned to a new column called `rating_category`.

You can also use anonymous functions as well. This lambda function achieves the same result as `rating_function`:

In [ ]:
movies_df["rating_category"] = movies_df["rating"].apply(lambda x: 'good' if x >= 8.0 else 'bad')

movies_df.head(2)

Overall, using `apply` will be much faster than iterating manually over rows because pandas is utilizing vectorization.

> Vectorization: a style of computer programming where operations are applied to whole arrays instead of individual elements —[Wikipedia](https://en.wikipedia.org/wiki/Vectorization)

A good example of high usage of `apply` is during natural language processing (NLP) work. You'll need to apply all sorts of text cleaning functions to strings to prepare for machine learning.

## Brief Plotting

Another great thing about pandas is that it integrates with Matplotlib, so you get the ability to plot directly off DataFrames and Series. To get started we need to import Matplotlib (`pip install matplotlib`), unless you're using Ananconda:

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20, 'figure.figsize': (10, 8)}) # set font and plot size to be larger

Now we can begin. There won't be a lot of coverage on plotting, but it should be enough to explore you're data easily.

**Side note:**
For categorical variables use Bar Charts and Boxplots.  For continuous variables utilize Histograms, Scatterplots, Line graphs, and Boxplots.

Let's plot the relationship between ratings and revenue. All we need to do is call `plot` on `movies_df` with some info about how to construct the plot:

In [ ]:
movies_df.plot(kind='scatter', x='rating', y='revenue_millions', title='Revenue (millions) vs Rating');

What's with the semicolon? It's not a syntax error, just a way to hide the `<matplotlib.axes._subplots.AxesSubplot at 0x26613b5cc18>` output when plotting in Jupyter notebooks.

If we want to plot a simple Histogram based on a single column, we can call plot on a column:

In [ ]:
movies_df['rating'].plot(kind='hist', title='Rating');

Do you remember the `describe` example at the beginning of this tutorial? Well, there's a graphical representation of the interquartile range, called the Boxplot. Let's recall what `describe` gives us on the ratings column:

In [ ]:
movies_df['rating'].describe()

Using a Boxplot we can visualize this data:

In [ ]:
movies_df['rating'].plot(kind="box");

<figure>
<img src="https://i1.wp.com/flowingdata.com/wp-content/uploads/2008/02/box-plot-explained.gif" />
    <figcaption>Source: <em>Flowing Data</em></figcaption>
</figure>
<br>

By combining categorical and continuous data, we can create a Boxplot of revenue that is grouped by the Rating Category we created above:

In [ ]:
movies_df.boxplot(column='revenue_millions', by='rating_category');

That's the general idea of plotting with pandas. There's too many plots to mention, so definitely take a look at the `plot` [docs here](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) for more information on what it can do.

## Wrapping up

Exploring, cleaning, transforming, and visualization data with pandas in Python is an essential skill in data science. Just cleaning wrangling data is 80% of your job as a Data Scientist. After a few projects and some practice, you should be very comfortable with most of the basics.

To keep improving, view the [extensive tutorials](https://pandas.pydata.org/pandas-docs/stable/tutorials.html) offered by the official pandas docs.

If you want to understand how pandas works step by step, take a look at the awesome [Pandas Tutor](https://pandastutor.com/vis.html) tool.